In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import time
import os
import cv2
import json
import matplotlib.pyplot as plt
from ipdb import set_trace
from glob import glob
from tqdm import tqdm

from model import model
from warp import WarpImageWithFlowAndBrightness

In [5]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [6]:
def process_json_list(json_list):
    ldmks = [eval(s) for s in json_list]
    return np.array([(x, y, z) for (x, y, z) in ldmks])

In [7]:
def get_filename_info(filename):
    info_dict = {}
    
    # Get image data
    img = cv2.cvtColor(cv2.imread("%s.jpg" % filename[:-5]), cv2.COLOR_BGR2RGB)
    # TODO: crop image
    img = cv2.resize(img, (64, 32))
    img = np.transpose(img, (2, 0, 1))
    info_dict['img'] = img
    

    # Get ID data
    titles_types = {'ID': int, 'T': str, 'N': int, 'F': int, 'V': float, 'H': float}
    info_list = os.path.basename(filename[:-5]).split('_')
    for title, info in zip(titles_types, info_list):
        info_dict[title] = titles_types[title](info[len(title):])
    info_dict['target'] = info_dict['F'] == 1
        
    # Get json data
    json_data_file = open(filename)
    json_data = json.load(json_data_file)
    info_dict["look_vec"] = np.array(eval(json_data["eye_details"]["look_vec"])) # 3D look direction vector
    info_dict["interior_margin"] = process_json_list(json_data["interior_margin_2d"]) # eye interior landmarks
    info_dict["caruncle"] = process_json_list(json_data["caruncle_2d"]) # caruncle landmarks
    info_dict["iris"] = process_json_list(json_data["iris_2d"]) # iris landmarks
    
    return info_dict

In [8]:
img_infos = []
input_file_path = os.path.join(os.getcwd(), '..', 'dataset', 'UnityEyes_Windows', 'imgs_1_cutouts')

json_fns = glob(os.path.join(input_file_path, "*.json"))
for json_fn in json_fns:
    info = get_filename_info(json_fn)
    img_infos.append(info)

In [9]:
img_df = pd.DataFrame(img_infos)
img_df.sort_values(['ID', 'F'], ignore_index=True, inplace=True)
img_df.head()

,img,ID,T,N,F,V,H,target,look_vec,interior_margin,caruncle,iris
0,"[[[97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97,...",1,gaussian,1,1,0.00,0.09,True,"[0.0016, 0.0, -1.0, 0.0]","[[352.8355, 265.1642, 9.0855], [349.2318, 278....","[[340.2054, 248.9607, 9.096], [333.4014, 259.0...","[[356.5728, 299.975, 8.9238], [357.405, 308.47..."
1,"[[[97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97,...",1,gaussian,1,2,-6.13,4.80,False,"[0.0832, -0.1068, -0.9908, 0.0]","[[352.9997, 264.6737, 9.0874], [349.6505, 277....","[[340.1695, 248.8329, 9.0979], [333.4014, 259....","[[363.3105, 290.1617, 8.8879], [363.9812, 298...."
2,"[[[97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97,...",1,gaussian,1,3,4.77,10.84,False,"[0.1874, 0.0831, -0.9788, 0.0]","[[352.6367, 265.7174, 9.0836], [348.7342, 279....","[[340.2437, 249.1077, 9.094], [333.4014, 259.0...","[[372.4717, 303.7707, 8.8355], [372.5953, 312...."
3,"[[[97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97,...",1,gaussian,1,4,-13.43,-5.19,False,"[-0.088, -0.2322, -0.9687, 0.0]","[[353.3353, 263.5461, 9.0922], [350.5352, 276....","[[340.0798, 248.548, 9.1021], [333.4014, 259.0...","[[349.5125, 282.5456, 9.007], [350.4209, 290.7..."
4,"[[[97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97,...",1,gaussian,1,5,-10.65,9.39,False,"[0.1603, -0.1848, -0.9696, 0.0]","[[353.2023, 264.0168, 9.0901], [350.1789, 276....","[[340.1184, 248.6655, 9.1004], [333.4014, 259....","[[369.9451, 282.7769, 8.8686], [370.5723, 291...."


### Notes on the dataframe info
- look_vec is a 3D homogeneous vector in the form \[x, y, z, 0\]. For purposes of the model input, only the x and y components are needed. I believe this vector is already normalized, but we can renormalize this vector before extracting the x and y components
- Feature landmarks (i.e., interior_margin, caruncle, and iris) are the pixel coordinates of feature BEFORE RESIZING. Ideally they are used to determine the crop/resize area


### Dataloader
The above dataframe contains info on all images, separated into IDs. As our model input, we would like to take all possible image pairs within an ID. With 40 images per ID, this results in 780 pairs.

Dataloader will output input image, input vector, target image, target vector

In [105]:
# Extract relevant data for dataloader
n_ids = img_df.iloc[-1]['ID']

imgs_list = []
vecs_list = []

# For each ID, generate all possible pairs
for id in range(1, n_ids + 1):
    # Get ID
    df_chunk = img_df.query(f"ID == {id}")
    imgs = np.stack(df_chunk['img'])
    vecs = np.stack(df_chunk['look_vec'].to_numpy())
    vecs = (vecs / np.linalg.norm(vecs, axis=1, keepdims=True))[:, :2] # normalize, then get x and y components
    vecs = np.tile(vecs[:, :, np.newaxis, np.newaxis], (1, 1, 32, 64))

    # Generate pairs
    pairs = np.triu_indices(len(df_chunk), k=1)
    pairs = np.stack(pairs).transpose()

    img_pairs = imgs[pairs] # (pair_idx, input or output, C, H, W)
    vec_pairs = vecs[pairs] # (pair_idx, input or output, x or y, ...)

    imgs_list.append(img_pairs)
    vecs_list.append(vec_pairs)

imgs_list = np.concatenate(imgs_list)
vecs_list = np.concatenate(vecs_list)

# Shuffle the lists
np.random.shuffle(imgs_list)
np.random.shuffle(vecs_list)

In [106]:
# Create splits
num_samples = len(imgs_list)
splits = [0.7, 0.2, 0.1]

X_img = imgs_list[:, 0, ...]
X_angle = vecs_list[:, 0, ...]
y_img = imgs_list[:, 1, ...]
y_angle = vecs_list[:, 1, ...]

X_img_train, X_img_valid, X_img_test = np.split(X_img, [int(num_samples * splits[0]), int(num_samples * (splits[0] + splits[1]))])
X_angle_train, X_angle_valid, X_angle_test = np.split(X_angle, [int(num_samples * splits[0]), int(num_samples * (splits[0] + splits[1]))])
y_img_train, y_img_valid, y_img_test = np.split(y_img, [int(num_samples * splits[0]), int(num_samples * (splits[0] + splits[1]))])
y_angle_train, y_angle_valid, y_angle_test = np.split(y_angle, [int(num_samples * splits[0]), int(num_samples * (splits[0] + splits[1]))])

In [107]:
X_img_train = torch.tensor(X_img_train, dtype=torch.int)
X_angle_train = torch.tensor(X_angle_train, dtype=torch.float32)
y_img_train = torch.tensor(y_img_train, dtype=torch.int)
y_angle_train = torch.tensor(y_angle_train, dtype=torch.int)

X_img_valid = torch.tensor(X_img_valid, dtype=torch.int)
X_angle_valid = torch.tensor(X_angle_valid, dtype=torch.float32)
y_img_valid = torch.tensor(y_img_valid, dtype=torch.int)
y_angle_valid = torch.tensor(y_angle_valid, dtype=torch.int)

X_img_test = torch.tensor(X_img_test, dtype=torch.int)
X_angle_test = torch.tensor(X_angle_test, dtype=torch.float32)
y_img_test = torch.tensor(y_img_test, dtype=torch.int)
y_angle_test = torch.tensor(y_angle_test, dtype=torch.int)

In [108]:
batch_size = 25
train_dataset = TensorDataset(X_img_train, X_angle_train, y_img_train, y_angle_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataset = TensorDataset(X_img_valid, X_angle_valid, y_img_valid, y_angle_valid)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_img_test, X_angle_test, y_img_test, y_angle_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [109]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
weight_correction_loss = 0.8
weight_reconstruction_loss = 0.2

printout_freq = 1
start_time = time.time()

warp = WarpImageWithFlowAndBrightness(next(iter(train_loader))[0])

train_losses, valid_losses = [], []

pbar = tqdm(range(num_epochs))
for epoch in pbar:
    epoch_start_time = time.time()
    model.train()
    train_loss = 0
    for imgs, angles, targets, target_angles in train_loader:
        imgs, angles, targets = imgs.float().to(device), angles.to(device), targets.float().to(device)
        
        flow_corr, bright_corr = model(imgs, angles)
        img_corr = warp(imgs, flow_corr, bright_corr)
        loss_correction = criterion(img_corr, targets)
        
        inverted_angles = -angles
        flow_reconstruction, bright_reconstruction = model(img_corr, inverted_angles)
        img_reconstruction = warp(img_corr, flow_reconstruction, bright_reconstruction)
        loss_reconstruction = criterion(img_reconstruction, imgs)
        
        loss = (weight_correction_loss * loss_correction) + (weight_reconstruction_loss) * loss_reconstruction
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_losses.append(train_loss / len(train_loader))
    
    model.eval()
    with torch.no_grad():
        valid_loss = 0
        for imgs, angles, targets, target_angles in valid_loader:
            imgs, angles, targets = imgs.float().to(device), angles.to(device), targets.float().to(device)

            flow_corr, bright_corr = model(imgs, angles)
            img_corr = warp(imgs, flow_corr, bright_corr)
            loss_correction = criterion(img_corr, targets)

            inverted_angles = -angles
            flow_reconstruction, bright_reconstruction = model(img_corr, inverted_angles)
            img_reconstruction = warp(img_corr, flow_reconstruction, bright_reconstruction)
            loss_reconstruction = criterion(img_reconstruction, imgs)

            loss = (weight_correction_loss * loss_correction) + (weight_reconstruction_loss) * loss_reconstruction
            valid_loss += loss.item()
        valid_losses.append(valid_loss / len(valid_loader))
    
    epoch_time = time.time() - epoch_start_time
    overall_time = time.time() - start_time
    num_days = int(overall_time / 86400)
    num_hrs = int((overall_time-(86400*num_days)) / 3600)
    num_mins = int((overall_time-(86400*num_days)-(3600*num_hrs)) / 60)
    num_secs = overall_time-(86400*num_days)-(3600*num_hrs)*(60*num_mins)
    if (epoch + 1) % printout_freq == 0:
    #     print(f"""Finished epoch {epoch + 1}/{num_epochs} ({(epoch+1)/num_epochs*100:.2f}%)
    #            total time is {num_days}d:{num_hrs}h:{num_mins}m:{num_secs:.3f}s; time for this epoch is {epoch_time:.2f}s
    #            training loss was {bcolors.BOLD}{train_loss:.3f}{bcolors.ENDC}, validation_loss was {bcolors.BOLD}{valid_loss:.3f}{bcolors.ENDC}.""")
        pbar.set_postfix({"TLoss": f"{train_loss:.3f}", "VLoss": f"{valid_loss:.3f}"})


model.eval()
test_loss = 0
with torch.no_grad():
    for imgs, angles, targets, target_angles in test_loader:
        imgs, angles, targets = imgs.float().to(device), angles.to(device), targets.float().to(device)

        flow_corr, bright_corr = model(imgs, angles)
        img_corr = warp(imgs, flow_corr, bright_corr)
        loss_correction = criterion(img_corr, targets)
        
        inverted_angles = -angles
        flow_reconstruction, bright_reconstruction = model(img_corr, inverted_angles)
        img_reconstruction = warp(img_corr, flow_reconstruction, bright_reconstruction)
        loss_reconstruction = criterion(img_reconstruction, imgs)
        
        loss = (weight_correction_loss * loss_correction) + (weight_reconstruction_loss) * loss_reconstruction
        test_loss += loss.item()
test_loss /= len(test_loader)
print(f'Test Loss: {test_loss}')

  0%|                                                                           | 0/100 [00:00<?, ?it/s]C:\Users\Codey\anaconda3\envs\unityeyes\Lib\site-packages\torch\nn\functional.py:4316: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
  0%|                                                                           | 0/100 [00:03<?, ?it/s]


KeyboardInterrupt: 